In [ ]:
!git clone --depth=1 https://github.com/twintproject/twint.git
%cd twint
!pip3 install . -r requirements.txt

In [ ]:
!pip install aiohttp==3.7.0

In [ ]:
pip install nest_asyncio

In [6]:
import nest_asyncio
nest_asyncio.apply()
import twint
import pandas as pd
import numpy as np
import re
import os

In [ ]:
c = twint.Config()
c.Username = 'ATCS_DISHUB_DIY'
c.Pandas = True
#c.Limit = 10
twint.run.Search(c)

In [8]:
Tweets_df = twint.storage.panda.Tweets_df
#Tweets_df.head()

In [9]:
Tweets_df = Tweets_df.drop(['conversation_id', 'created_at', 'timezone', 'id', 'place', 'language', 'hashtags', 'cashtags', 'user_id', 'user_id_str', 'username', 'name', 'day', 'hour', 'urls', 'video', 'thumbnail', 'retweet', 'nlikes', 'nreplies', 'nretweets', 'quote_url', 'search', 'near', 'geo', 'source', 'user_rt_id', 'user_rt', 'retweet_id', 'reply_to', 'retweet_date', 'translate', 'trans_src', 'trans_dest'], axis = 1)
Tweets_df = Tweets_df.dropna(axis = 1)

In [10]:
Tweets_df.to_excel('CrawlTwint.xlsx', index = False)

In [11]:
#from google.colab import files
#files.download("CrawlTwint.xlsx")

In [ ]:
Tweets_df['date'] = Tweets_df['date'].str.replace(r'\D', '', case=False)

In [ ]:
print(Tweets_df['date'])

In [14]:
convString = pd.to_datetime(Tweets_df['date'], format='%Y%m%d%H%M%S')

In [ ]:
print(convString)

In [16]:
Tweets_df['date'] = convString.dt.tz_localize('UTC').dt.tz_convert('Asia/Jakarta')

In [ ]:
print(Tweets_df['date'])

In [18]:
Tweets_df['date'] = Tweets_df['date'].dt.tz_localize(None)
Tweets_df['date'] = Tweets_df['date'].astype(str)
Tweets_df.to_excel('TwintTimeCleaning.xlsx', index = False)

In [19]:
#from google.colab import files
#files.download("TwintTimeCleaning.xlsx")

In [ ]:
key_value = ["jalanan",
    "lalin",
    "lalu lintas",
    "lancar",
    "macet",
    "padat",
    "ramai",
    "tersendat"]
df = pd.read_excel('TwintTimeCleaning.xlsx')
df = df[df['tweet'].str.contains('|'. join(key_value))]
df['tweet'] = df['tweet'].str.replace('http\S+|www.\S+', '', case=False)
df['tweet'] = df['tweet'].str.replace('&amp;', 'dan', case=False)

In [ ]:
df

In [22]:
df.to_excel('TwintFullClean.xlsx', index = False)

In [ ]:
dfProcess = pd.read_excel('TwintFullClean.xlsx')
dfStatus = pd.read_excel('DataNonJalan.xlsx')
#dfJalan = pd.read_excel('DataSimpangDIY.xlsx')
dfJalan = pd.read_excel('DataSimpangDIYNormal.xlsx')

listKondisi = dfStatus['Kondisi'].tolist()
listKondisi = [x for x in listKondisi if pd.isnull(x) == False]

listKeterangan = dfStatus['Peristiwa'].tolist()
listKeterangan = [x for x in listKeterangan if pd.isnull(x) == False]

listCuaca = dfStatus['Cuaca'].tolist()
listCuaca = [x for x in listCuaca if pd.isnull(x) == False]

dfProcess = dfProcess[dfProcess['tweet'].str.contains('|'. join(listKondisi))]

print(listKondisi)
print(listKeterangan)
print(listCuaca)

In [ ]:
timeConv = pd.to_datetime(dfProcess['date'])
timeConv = timeConv.reset_index(drop=True)
dftimeConv = timeConv.dt.strftime('%Y-%b-%d')
dftimeYear = timeConv.dt.strftime('%Y')
dftimeMonth = timeConv.dt.strftime('%B')
dftimeDay = timeConv.dt.day_name()
print(dftimeDay)
print(dftimeMonth)
print(dftimeYear)
print(dftimeConv)
print(timeConv)

In [ ]:
# simpangList = dfJalan['simpang'].tolist()
# arahList = dfJalan['arah'].tolist()
simpangList = dfJalan['Simpang'].tolist()
arahList = dfJalan['Arah'].tolist()

simpangList = [x for x in simpangList if pd.isnull(x) == False]
arahList = [x for x in arahList if pd.isnull(x) == False]
streetList = simpangList + arahList
streetList = list(dict.fromkeys(streetList))
print(streetList)

In [ ]:
dfInsert = dfProcess.insert(1,"Date", np.nan)
dfInsert = dfProcess.insert(2,"Year", np.nan)
dfInsert = dfProcess.insert(3,"Month", np.nan)
dfInsert = dfProcess.insert(4,"Day", np.nan)
dfInsert = dfProcess.insert(5,"Waktu", np.nan)
dfInsert = dfProcess.insert(6,"Dari", np.nan)
dfInsert = dfProcess.insert(7,"Arah", np.nan)
dfInsert = dfProcess.insert(8,"Kondisi", np.nan)
dfInsert = dfProcess.insert(10,"Hambatan", np.nan)
dfProcess = dfProcess.reset_index(drop=True)
dftimeConv = dftimeConv.reset_index(drop=True)
dfProcess

In [ ]:
tweet = dfProcess['tweet'].tolist()

listDate = []
listTime = []
listFrom = []
listTo = []
listCond = []
listCause = []

for n in range(0, len(tweet), +1):
  newList = tweet[n]
  newStr = newList

  fixedStr = newStr
  strToReplace = {'gangguan mesin': 'mogok', 'demo': 'demonstrasi', 
                  'sp':'simpang', 'disimpang':'simpang', 'berlawanan':'sebaliknya', 'segala arah':'',
                  'kidsfun':'Kidsfun', 'kids fun':'Kidsfun', 'Kids fun':'Kidsfun', 'Kids Fun':'Kidsfun',
                  'jatikencana':'Jati Kencana', 'Jatikencana':'Jati Kencana',
                  'pelemgurih':'Pelem Gurih', 'pelem gurih':'Pelem Gurih', 'Pelem gurih':'Pelem Gurih', 'pelem Gurih':'Pelem Gurih',
                  'blok o':'Blok O', 'blok O':'Blok O', 'Blok o':'Blok O', 'BLOK O':'Blok O', 'BLOK o':'Blok O',
                  'maguwo':'Maguwoharjo', 'Maguwo':'Maguwoharjo', 'MAGUWO':'Maguwoharjo', 'maguwoharjo':'Maguwoharjo', 'MAGUWOHARJO':'Maguwoharjo', 'maguwo harjo':'Maguwoharjo', 'Maguwo harjo':'Maguwoharjo', 'maguwo Harjo':'Maguwoharjo', 'Maguwo Harjo':'Maguwoharjo',
                  'kotagedhe':'Kota Gedhe', 'Kotagedhe':'Kota Gedhe', 'kotaGedhe':'Kota Gedhe', 'KOTAgedhe':'Kota Gedhe', 'kotaGEDHE':'Kota Gedhe', 'kota gedhe':'Kota Gedhe', 'kota Gedhe':'Kota Gedhe', 'KOTA gedhe':'Kota Gedhe', 'kota GEDHE':'Kota Gedhe', ''
                  'mm ugm':'MM UGM', 'MM ugm':'MM UGM', 'mmugm':'MM UGM', 'mm UGM':'MM UGM', 'MMugm':'MM UGM', 'mmUGM':'MM UGM', 'MMUGM':'MM UGM',
                  'tempel':'Tempel', 'giwangan':'Giwangan', 'gamping':'Gamping', 'demangan':'Demangan', 'bandara':'Bandara',
                  'seturan':'Seturan', 'ngasem':'Ngasem', 'wojo':'Wojo', 'uin':'UIN', 'pogung':'Pogung',
                  'kentungan':'Kentungan', 'tugu':'Tugu', 'gramedia':'Gramedia', 'galeria':'Galeria',
                  'condongcatur':'Condongcatur', 'CondongCatur':'Condongcatur',
                  'druwo':'Druwo',
                  'patran':'Patran',
                  'timur':'Timur',
                  'barat':'Barat',
                  'selatan':'Selatan',
                  'utara':'Utara'}
  for key, value in strToReplace.items():
      fixedStr = fixedStr.replace(key, value)
      fixedStr = re.sub("[\(\[].*?[\)\]]", "", fixedStr)

#>>>Proses splitting arah jalan dan splitting waktu<<<
  splitList = fixedStr.split('menuju' and 'arah' or 'ke')
  print(splitList)
  # lenKond = len(splitKond)
  # print(splitKond)

  tweetTime = dfProcess['date'].tolist()
  timeTweet = tweetTime[n]
  splitTime = timeTweet.split(' ')
  #print(splitTime)

  lenList = len(splitList)
  emptyStr = '-'

  tglTweet = dftimeConv[n]
  waktuTweet = splitTime[1]

  #print('Date: ' + tglTweet)
  #print('Tweet Time: ' + waktuTweet)

  print('Tweet: ' + newList)

#>>>Start proses ekstraksi teks dari twitter<<<
  if (lenList > 1):
    fromList = splitList[0]
    fromWord = [word for word in streetList if word in fromList]
    toList = splitList[1]
    toWord = [word for word in streetList if word in toList]
    fromStr = ", ".join(fromWord)
    toStr = ", ".join(toWord)
    print('From: ' + fromStr)
    print('To: ' + toStr)
  else:
    fromList = splitList[0]
    fromWord = [word for word in streetList if word in fromList]
    fromStr = " ".join(fromWord)
    toStr = emptyStr
    print('From: ' + fromStr)
    print('To: ' + toStr)

  tweetKondisi = [x for x in listKondisi if x in newList]
  kondisiStr = " ".join(tweetKondisi)
  if 'ramai lancar' in newList:
    kondisiStr = 'ramai lancar'
  elif 'lancar' in newList:
    kondisiStr = 'lancar'
  kondisiStrrev = emptyStr
  # print('Condition: ' + kondisiStr)

  # if (lenKond > 1):
  #   kondList = splitKond[1]
  #   tweetKondisirev = [x for x in listKondisi if x in kondList]
  #   kondisiStrrev = " ".join(tweetKondisirev)
  #   if 'ramai lancar' in kondList:
  #     kondisiStrrev = 'ramai lancar'
  #   elif 'lancar' in kondList:
  #     kondisiStrrev = 'lancar'
  #   print('Opposite: ' + kondisiStrrev)

  emptyList = []
  tweetKeterangan = [x for x in listKeterangan if x in fixedStr]
  if (tweetKeterangan == emptyList):
    keteranganStr = emptyStr
  else:
    keteranganStr = " ".join(tweetKeterangan)
  #print('Cause: ' + keteranganStr)

  # tweetCuaca = [x for x in listCuaca if x in fixedStr]
  # if (tweetCuaca == emptyList):
  #   cuacaStr = emptyStr
  # else:
  #   cuacaStr = " ".join(tweetCuaca)
  # #print('Weather: ' + cuacaStr)
  # #print('\n')

  listDate.append(tglTweet)
  listTime.append(waktuTweet)
  listFrom.append(fromStr)
  listTo.append(toStr)
  listCond.append(kondisiStr)
  listCause.append(keteranganStr)

#>>>End proses ekstraksi teks dari twitter<<<

#print(listFrom)
print(listTo)
#print(listCond)
#print(listCause)
#print(listWeather)

#>>>Memasukkan hasil proses ekstraksi ke tabel<<<
dfFinalTweet = dfProcess.assign(Date = dftimeConv, Waktu = listTime, Year = dftimeYear, Month = dftimeMonth, Day = dftimeDay, Dari = listFrom, Arah = listTo, Kondisi = listCond, Hambatan = listCause)
dfFinalTweet = dfFinalTweet.drop(['date'], axis=1)
dfFinalTweet = dfFinalTweet.rename(columns = {'tweet':'Tweet'})
print(dfFinalTweet)

In [29]:
dfFinalTweet.to_excel('DatasetTweetFinal.xlsx', index = False)

In [30]:
dfFinalTweet.to_csv('DatasetTweetFinal.csv', index = False)

In [31]:
# read DataFrame
data = pd.read_csv("DatasetTweetFinal.csv")
!mkdir datasets
root = 'datasets'
for (Date), group in data.groupby(['Date']):
     group.to_csv(root + '/' + f'{Date}.csv', index=False)

# print(pd.read_csv("2022-Sep-22.csv"))
# print(pd.read_csv("2022-Sep-17.csv"))

In [93]:
docMaster = []
for file in os.listdir('datasets'):
  if file.endswith ('.csv'):
    docMaster.append (pd.read_csv(os.path.join('datasets', file)))

In [ ]:
docMain = pd.concat(docMaster, axis=0)
dfdocMain = docMain.to_csv('DocFull.csv', index = True)
dfdocMain = pd.read_csv('DocFull.csv')
dfdocMain = dfdocMain.dropna(axis = 0)

dfdocMain = dfdocMain.apply(lambda x: x.replace(strToReplace, regex=True))
listdocMain = dfdocMain['Tweet'].tolist()
print(dfdocMain)
print(listdocMain)

# for n in range(0, len(listdocMain), +1):
#   newListdoc = tweet[n]
#   newStrdoc = newListdoc
#   fixedStr = newStrdoc
#   for key, value in strToReplace.items():
#       fixedStr = fixedStr.replace(key, value)
#       fixedStr = re.sub("[\(\[].*?[\)\]]", "", fixedStr)
#       # dfdocMain['Tweet'] = fixedStr
#       listfixedStr = list(fixedStr.split(" "))
#       df = df[df['tweet'].str.contains('|'. join(key_value))]
# print(listfixedStr)
# print(fixedStr)
# print(newListdoc)
# print(listdocMain)

In [108]:
docs = doc1 + doc2 + doc3 + doc4 + doc5
arrdocs = np.array(listdocMain)

In [96]:
streetList2 = simpangList + arahList

In [97]:
streetSet = set(streetList2)
arrStreet = np.array(streetList2)

In [98]:
# inverted_index = {}

# for i, streetSet in enumerate(docs):
#   for term in streetSet:
#     if term in inverted_index:
#       inverted_index[term].add(i)
#     else: inverted_index[term] = {i}

# inverted_index

In [ ]:
# print(arrdocs)
invIndex = {}
for i in range(len(dfdocMain)):
    check = arrdocs[i]
    for item in arrStreet:
        if item in check:
            if item not in invIndex:
                invIndex[item] = []
            if item in invIndex:
                invIndex[item].append(i+1)

invIndex

In [100]:
# posting_list = inverted_index['Seturan']
# posting_list

In [101]:
# import os
# path = 'datasets'
# filenames = os.listdir(path)
# # print(filenames)

# def is_name_in_csv(word,csv_file):
#   with open(csv_file,"r") as f:
#     data = f.read()
#   return bool(re.search(word, data))

# list_file_csv = filenames
# word = ('|'. join(streetList))
# for csv_file in list_file_csv:
#   if is_name_in_csv(word,csv_file):
#     print(f"Found {word} in {csv_file}")
#   else:
#     print(f"Finished. Can't find any more data with that word")
#     break

In [ ]:
from collections import defaultdict
inv_index = defaultdict(list)
for doc, words in zip(
        dfdocMain['Date'],
        dfdocMain['Dari'].str.findall('|'. join(listFrom)).map(set)):
    for word in words:
        inv_index[word].append(doc)
print(inv_index)

In [103]:
# print(tweet)
# arr_tweet = np.array(tweet)
# arr_streetList = np.array(streetList)
# print(arr_streetList)
# #print(arr_tweet)
# dict = {}

# for i in range(len(tweet)):
#     check = arr_tweet[i].lower()
#     for item in arr_streetList:
#         if item in check:
#             if item not in dict:
#                 dict[item] = []
#             if item in dict:
#                 dict[item].append(i+1)

# dict

In [ ]:
dfFinish = pd.read_excel('DatasetTweetFinal.xlsx')
dfFinish = dfFinish.dropna(axis=0)
fixedListFrom = list(dict.fromkeys(listFrom))
# print(fixedListFrom)
greet = "Hi! Saya akan mengambil tweet Jakarta Utara dari TMCPoldaMetroJaya. Coba tulis nama jalannya disini ya!"
print(greet)

def posindexing(inputstr):
  from collections import defaultdict
  ind = inputstr
  pos_index = defaultdict(list)
  for doc, words in zip(
      dfdocMain['Date'],
      dfdocMain['Dari'].str.findall(ind).map(set)):
    for word in words:
        pos_index[word].append(doc)
  print(pos_index)

def indexing(inputstr):
  invIndex = {}
  for i in range(len(dfdocMain)):
      check = arrdocs[i]
      for item in arrStreet:
          if item in check:
              if item not in invIndex:
                  invIndex[item] = []
              if item in invIndex:
                  invIndex[item].append(i+1)
  x = invIndex[inputstr]
  print(x)

def start():
  print('Coba masukkan nama jalan ')
  inputstr = input('')
  checkWord = [word for word in fixedListFrom if word in inputstr]
  fixedInput = ''.join(checkWord)
  print(checkWord)
  dfBot = dfFinish[dfFinish['Dari'].str.contains(inputstr)]
  posindexing(inputstr)
  indexing(inputstr)
  print('\n')

  showDate = dfBot['Date'].tolist()
  showYear = dfBot['Year'].tolist()
  showMonth = dfBot['Month'].tolist()
  showDay = dfBot['Day'].tolist()
  showTime = dfBot['Waktu'].tolist()
  showTweet = dfBot['Tweet'].tolist()
  showFrom = dfBot['Dari'].tolist()
  showTo = dfBot['Arah'].tolist()
  showCond = dfBot['Kondisi'].tolist()
  showCause = dfBot['Hambatan'].tolist()
  showAll = dfBot.values.tolist()
  showAll = sum(showAll, [])
  showAll = ' '.join(map(str, showAll))

  if inputstr in showAll:
    print("Yay! Saya menemukan beberapa Tweet!\n")
    for n in range(0, len(showTweet), +1):
      print('Date: ' + str(showDate[n]))
      print('Year: ' + str(showYear[n]))
      print('Month: ' + showMonth[n])
      print('Day: ' + showDay[n])
      print('Time: ' + showTime[n])
      print('Tweet: ' + showTweet[n])
      print('From: ' + showFrom[n])
      print('To: ' + showTo[n])
      print('Condition: ' + showCond[n])
      print('Cause: ' + str(showCause[n]))

      print('\n')

  else:
    print('Hmm... Sepertinya TMC Polda Metro Jaya belum Tweet jalan itu. Bagaimana dengan jalan lainnya?')
    start()
start()